In [5]:
import geopandas as gpd
from sqlalchemy import create_engine
import os
import pandas as pd
from POSTGRES import dbname, user, password, host, port

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

# folder_path = "../data"
# geojson_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.geojson')]
wojewodztwa = "../granice_jednostek/granice_wojewodztw.shp"
# powiaty = "../granice_jednostek/granice_powiatow.shp"
# gminy = "../granice_jednostek/granice_gmin.shp"

woj_gdf = gpd.read_file(wojewodztwa)
print (woj_gdf.columns)
# pow_gdf = gpd.read_file(powiaty)
# gmi_gdf = gpd.read_file(gminy)

# pow_gdf['woj_kod'] = pow_gdf['JPT_KOD_JE'].str[:2]

# # Gminy: dołącz kolumnę z kodem powiatu (pierwsze 4 znaki)
# gmi_gdf['pow_kod'] = gmi_gdf['JPT_KOD_JE'].str[:4]

# total_objects = 0
# gdfs = []

# for file in geojson_files:
#     temp_gdf = gpd.read_file(file)
#     print(f"Plik: {file}, crs: {temp_gdf.crs}")
#     gdfs.append(temp_gdf)

# gdf = pd.concat(gdfs, ignore_index=True)
# print(len(gdf))

Index(['gml_id', 'JPT_KOD_JE', 'JPT_SJR_KO', 'JPT_NAZWA_', 'JPT_POWIER',
       'JPT_ORGAN_', 'JPT_KOD__1', 'JPT_NAZWA1', 'JPT_ORGAN1', 'JPT_WAZNA_',
       'IIP_PRZEST', 'IIP_IDENTY', 'JPT_KJ_IIP', 'JPT_KJ_I_1', 'JPT_KJ_I_2',
       'JPT_SPS_KO', 'JPT_POWI_1', 'JPT_KJ_I_3', 'JPT_JOR_ID', 'WERSJA_OD',
       'WERSJA_DO', 'WAZNY_OD', 'WAZNY_DO', 'IIP_WERSJA', 'JPT_OPIS', 'JPT_ID',
       'REGON', 'SHAPE_LENG', 'SHAPE_AREA', 'geometry'],
      dtype='object')


In [2]:
# gdf = gdf.reset_index(drop=True)
# gdf['id'] = gdf.index + 1

# gdf.to_postgis("zdjecia_lotnicze", engine, if_exists="replace", index=False)
# woj_gdf.to_postgis("wojewodztwa", engine, if_exists="replace", index=False)
# pow_gdf.to_postgis("powiaty", engine, if_exists="replace", index=False)
# gmi_gdf.to_postgis("gminy", engine, if_exists="replace", index=False)

print("Dane zostały zapisane do bazy!")


Dane zostały zapisane do bazy!


In [ ]:
from sqlalchemy import text

with engine.begin() as conn:
    # conn.execute(text("CREATE INDEX IF NOT EXISTS idx_wojewodztwa_geom ON wojewodztwa USING GIST (geometry);"))
    # conn.execute(text("CREATE INDEX IF NOT EXISTS idx_powiaty_geom ON powiaty USING GIST (geometry);"))
    # conn.execute(text("CREATE INDEX IF NOT EXISTS idx_gminy_geom ON gminy USING GIST (geometry);"))
    print("Indeksy zostały utworzone.")
    conn.execute(text("ALTER TABLE zdjecia_lotnicze DROP COLUMN IF EXISTS wojt_kod;"))
    conn.execute(text("ALTER TABLE zdjecia_lotnicze ADD COLUMN IF NOT EXISTS woj_jpt_kod TEXT;"))
    conn.execute(text("ALTER TABLE zdjecia_lotnicze ADD COLUMN IF NOT EXISTS pow_jpt_kod TEXT;"))
    conn.execute(text("ALTER TABLE zdjecia_lotnicze ADD COLUMN IF NOT EXISTS gmi_jpt_kod TEXT;"))

    # conn.execute(text("VACUUM ANALYZE zdjecia_lotnicze;"))
    # conn.execute(text("VACUUM ANALYZE wojewodztwa;"))
    # conn.execute(text("VACUUM ANALYZE powiaty;"))
    # conn.execute(text("VACUUM ANALYZE gminy;"))
    # print("VACUUM ANALYZE zostało wykonane.")

    conn.execute(text("ALTER TABLE zdjecia_lotnicze ADD PRIMARY KEY (id);"))

    # conn.execute(text("""
    #     ALTER TABLE zdjecia_lotnicze
    #     ADD COLUMN IF NOT EXISTS woj_jpt_kod text,
    #     ADD COLUMN IF NOT EXISTS pow_jpt_kod text,
    #     ADD COLUMN IF NOT EXISTS gmi_jpt_kod text;
    # """))

    print("Dodano kolumny woj_jpt_kod, pow_jpt_kod i gmi_jpt_kod do tabeli zdjecia_lotnicze.")
    conn.execute(text("""
        UPDATE zdjecia_lotnicze z
        SET woj_jpt_kod = w."JPT_KOD_JE"
        FROM wojewodztwa w
        WHERE ST_Intersects(z.geometry, w.geometry);
    """))
    print("Zaktualizowano kolumnę woj_jpt_kod w tabeli zdjecia_lotnicze.")
    conn.execute(text("""
        UPDATE zdjecia_lotnicze z
        SET pow_jpt_kod = p."JPT_KOD_JE"
        FROM powiaty p
        WHERE ST_Intersects(z.geometry, p.geometry);
    """))
    print("Zaktualizowano kolumnę pow_jpt_kod w tabeli zdjecia_lotnicze.")
    conn.execute(text("""
        UPDATE zdjecia_lotnicze z
        SET gmi_jpt_kod = g."JPT_KOD_JE"
        FROM gminy g
        WHERE ST_Intersects(z.geometry, g.geometry);
    """))
    print("Zaktualizowano kolumnę gmi_jpt_kod w tabeli zdjecia_lotnicze.")



Indeksy zostały utworzone.
Dodano kolumny woj_jpt_kod, pow_jpt_kod i gmi_jpt_kod do tabeli zdjecia_lotnicze.
Zaktualizowano kolumnę woj_jpt_kod w tabeli zdjecia_lotnicze.
Zaktualizowano kolumnę pow_jpt_kod w tabeli zdjecia_lotnicze.
Zaktualizowano kolumnę gmi_jpt_kod w tabeli zdjecia_lotnicze.


In [12]:
from sqlalchemy import create_engine, text

# Połączenie z bazą
engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{dbname}")

with engine.begin() as conn:  # zamiast engine.connect()
    conn.execute(text("""
        ALTER TABLE zdjecia_lotnicze
        ADD COLUMN IF NOT EXISTS wojt_kod text,
        ADD COLUMN IF NOT EXISTS pow_jpt_kod text,
        ADD COLUMN IF NOT EXISTS gmi_jpt_kod text;
    """))